In [1]:
using JuMP
using Gurobi
using MixedIntegerExperiments
using Polyhedra
# using BenchmarkTools
# using Plots
# using DrakeVisualizer

In [2]:
# Pkg.clone("https://github.com/andreasnoack/LinearAlgebra.jl")
# Pkg.clone("git@github.com:JuliaPolyhedra/ConvexHull.jl.git")

In [3]:
# Pkg.checkout("DrakeVisualizer", "bugfixes")

In [4]:
verbose = true;

In [5]:
# parameters
nsteps = 100
ncontacts = 2
h = 0.1
I = 1.
m = 10.
g = [0; -9.81]
r0 = [0.5; 0.8]
ṙ0 = [0.0; 0.0]
θ0 = 0.
ω0 = 0.
contactregion = contact_region([0.; 0.], [1.; 0.], 0.8, 2 * m * norm(g))
freeregion = axis_aligned_free_box_region([0.; 0.], [1.; 1.])
regions = [contactregion; freeregion]
nregions = length(regions);

In [6]:
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
# Visualizer(polyhedron(contactregion.position))

LoadError: UndefVarError: DrakeVisualizer not defined

In [7]:
model = Model(solver=GurobiSolver(Presolve=0, OutputFlag=Int(verbose)));

In [8]:
@variables(model, begin
    z[i = 1 : ncontacts, j = 1 : nregions, n = 1 : nsteps], Bin
    r̂[k = 1 : 2, i = 1 : ncontacts, j = 1 : nregions, n = 1 : nsteps]
    f̂[k = 1 : 2, i = 1 : ncontacts, j = 1 : nregions, n = 1 : nsteps]
    r[k = 1 : 2, n = 1 : nsteps]
    θ[n = 1 : nsteps]
    ṙ[k = 1 : 2, n = 1 : nsteps]
    ω[n = 1 : nsteps]
    rc[k = 1 : 2, i = 1 : ncontacts, n = 1 : nsteps]
    f[k = 1 : 2, i = 1 : ncontacts, n = 1 : nsteps]
    F[k = 1 : 2, n = 1 : nsteps]
    T[n = 1 : nsteps]
    s[k = 1 : 2, i = 1 : ncontacts, n = 1 : nsteps]
    wzx[i = 1 : ncontacts, n = 1 : nsteps]
    wxz[i = 1 : ncontacts, n = 1 : nsteps]
    γ[i = 1 : ncontacts, j = 1 : nregions, n = 1 : nsteps - 1]
    Γ[i = 1 : ncontacts, j = 1 : nregions]
end)

In [9]:
function hrep_to_constraints(model::Model, hr::HRepresentation, vars::Vector{Variable})
    for ineq in ineqs(hr)
        @constraint(model, ineq.a ⋅ vars <= ineq.β)
    end
    for eq in eqs(hr)
        @constraint(model, eq.a ⋅ vars == eq.β)
    end
end

hrep_to_constraints (generic function with 1 method)

In [10]:
# 4.6
for j = 1 : nregions
    region = regions[j]
    for i = 1 : ncontacts, n = 1 : nsteps
        hrep_to_constraints(model, region.position, r̂[:, i, j, n])
        hrep_to_constraints(model, region.force, f̂[:, i, j, n])
    end
    
    lr, ur = axis_aligned_bounding_box(region.position)
    lf, uf = axis_aligned_bounding_box(region.force)
    @constraints(model, begin
        [i = 1 : ncontacts, k = 1 : 2, n = 1 : nsteps], z[i, j, n] * lr[k] <= r̂[k, i, j, n]
        [i = 1 : ncontacts, k = 1 : 2, n = 1 : nsteps], r̂[k, i, j, n] <= z[i, j, n] * ur[k]
        [i = 1 : ncontacts, k = 1 : 2, n = 1 : nsteps], z[i, j, n] * lf[k] <= f̂[k, i, j, n]
        [i = 1 : ncontacts, k = 1 : 2, n = 1 : nsteps], f̂[k, i, j, n] <= z[i, j, n] * uf[k]
    end)
end

@constraints(model, begin
    [i = 1 : ncontacts, k = 1 : 2, n = 1 : nsteps], sum(r̂[k, i, j, n] for j = 1 : nregions) == rc[k, i, n]
    [i = 1 : ncontacts, k = 1 : 2, n = 1 : nsteps], sum(f̂[k, i, j, n] for j = 1 : nregions) == f[k, i, n]
    [i = 1 : ncontacts, n = 1 : nsteps], sum(z[i, j, n] for j = 1 : nregions) == 1
end)

In [11]:
@constraints(model, begin
    [n = 1 : nsteps, i = 1 : ncontacts, k = 1 : 2], s[k, i, n] == rc[k, i, n] - r[k, n]
end)

In [12]:
# 4.18
@constraints(model, begin
    [n = 1 : nsteps - 1, k = 1 : 2], r[k, n + 1] == r[k, n] + h * ṙ[k, n + 1]
    [n = 1 : nsteps - 1], θ[n + 1] == θ[n] + h * ω[n + 1]
    [n = 1 : nsteps - 1, k = 1 : 2], ṙ[k, n + 1] == ṙ[k, n] + h / m * F[k, n + 1]
    [n = 1 : nsteps - 1], ω[n + 1] == ω[n] + h / I * T[n + 1]
    [n = 1 : nsteps, k = 1 : 2], F[k, n] == m * g[k] + sum(f[k, i, n] for i = 1 : ncontacts)
    [n = 1 : nsteps], T[n] == sum(wzx[i, n] - wxz[i, n] for i = 1 : ncontacts)
end)

In [13]:
# function mccormick_envelope_constraints(model::Model, w::Variable, u::Variable, v::Variable, urange::FloatRange)
#     # convex approximation of w == u * v
#     # 4.21
#     @assert first(urange) >= 0
#     @assert last(urange) <= 1
#     @constraints(model, begin
#         w >= first(urange) * v
#         w >= last(urange) * v + u - last(urange)
#         w <= first(urange) * v + u - first(urange)
#         w <= last(urange) * v
#         first(urange) <= u
#         u <= last(urange)
#     end)
# end

In [14]:
function mccormick_envelope_constraints(model::Model, w::Variable, u::Variable, v::Variable, urange::FloatRange)
    @assert first(urange) == 0. && last(urange) == 1.
    @constraints(model, begin
        w <= v
        w <= u
        w >= u + v - 1
        0 <= u
        u <= 1
        0 <= v
        v <= 1
#         0 <= w
#         w <= 1
    end)
end

mccormick_envelope_constraints (generic function with 1 method)

In [15]:
# convex bilinear term approximations (4.12, 4.13)
for i = 1 : ncontacts, n = 1 : nsteps
    mccormick_envelope_constraints(model, wxz[i, n], s[1, i, n], f[2, i, n], 0. : 1.) # TODO: range
    mccormick_envelope_constraints(model, wzx[i, n], s[2, i, n], f[1, i, n], 0. : 1.)
end

In [16]:
# initial conditions
@constraints(model, begin
    r[:, 1] .== r0
    θ[1] == θ0
    ṙ[:, 1] .== ṙ0
    ω[1] == ω0
end)

In [17]:
# TODO: more task-specific constraints
@constraints(model, begin
    [n = 1 : nsteps], r[2, n] >= 0.
end)

In [18]:
# objective function + slack parameterization
@constraints(model, begin
    [i = 1 : ncontacts, j = 1 : nregions], Γ[i, j] == sum(γ[i, j, n] for n = 1 : nsteps - 1)
    [i = 1 : ncontacts, j = 1 : nregions, n = 1 : nsteps - 1], -γ[i, j, n] <= z[i, j, n + 1] - z[i, j, n]
    [i = 1 : ncontacts, j = 1 : nregions, n = 1 : nsteps - 1], z[i, j, n + 1] - z[i, j, n] <= γ[i, j, n]
    [i = 1 : ncontacts, j = 1 : nregions, n = 1 : nsteps - 1], 0 <= γ[i, j, n]
    [i = 1 : ncontacts, j = 1 : nregions, n = 1 : nsteps - 1], γ[i, j, n] <= 1
end)

# @objective(model, begin
#     sum(Γ[i, j] for i = 1 : ncontacts, j = 1 : nregions) + sum(f[]) # TODO
# end)

In [19]:
solve(model)

Optimize a model with 12588 rows, 4900 columns and 22156 nonzeros
Coefficient statistics:
  Matrix range    [1e-02, 2e+02]
  Objective range [0e+00, 0e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [5e-01, 2e+02]
Variable types: 4500 continuous, 400 integer (400 binary)

Root relaxation: infeasible, 4108 iterations, 0.52 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0               - infeasible      -     -    0s

Explored 0 nodes (4108 simplex iterations) in 0.59 seconds
Thread count was 4 (of 8 available processors)

Model is infeasible
Best objective -, best bound -, gap -


:Infeasible